In [4]:
import pandas as pd
import datetime
import numpy as np
import time

## Grab Transfer data and remove unneeded fields and convert dates to data datatype

In [14]:
df = pd.read_csv('transfers.csv')
df = df.rename(columns = {'Start_Bed_Dttm':'Start', 'End_Bed_Dttm':'End'})
df = df.drop(columns=['Transfers', 'Bed_Code', 'Full Flow Code', 'Current Flow Code', '1st Level','PASID'])
date_cols = ['Start', 'End']
df[date_cols] = df[date_cols].apply(pd.to_datetime, format='%d/%m/%y %H:%M', errors='raise')
#df[date_cols] = df[date_cols].apply(lambda x: x.dt.strftime('%d/%m/%Y %H:%M:%S'))
df

,Link,Ward_Code,Start,End
0,ADE-363169,MT,2016-02-22 05:30:00,2016-02-27 11:45:00
1,ADE-271165,MT,2013-06-30 22:21:00,2013-07-04 10:40:00
2,ADE-461781,ICU,2019-05-09 18:17:00,2019-05-10 13:24:00
3,ADE-501083,DS,2020-06-20 20:00:00,2020-06-22 03:38:00
4,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00
...,...,...,...,...
8921,ADE-348899,MT,2016-02-15 14:27:00,2016-02-19 10:15:00
8922,ADE-491342,DS,2019-12-15 20:00:00,2019-12-16 13:24:00
8923,ADE-348904,MT,2016-04-05 22:51:00,2016-04-09 09:45:00
8924,ADE-425431,DS,2018-01-29 20:30:00,2018-01-31 11:27:00


## Determine start and finish dates and create a timestamp table to draw in patient counts at each time stamp

In [3]:
data_start = df['Start'].min()
data_end = df['End'].max()
data_period = data_end - data_start
data_period_minutes = (data_period.total_seconds())/60

interval_period = 20 #minutes
intervals = int(round(data_period_minutes / interval_period,0)) #No of 15 minute intervals

timestamp = []
time_interval = data_start
for i in range(1, intervals):
    timestamp.append(time_interval)
    interval_length = datetime.timedelta(minutes = interval_period)
    time_interval = data_start + i * interval_length

blank_timestamp_dataset = {'Time': timestamp,
        'MT Patients': 0,
         'DS Patients': 0}
pcount_df = pd.DataFrame(blank_timestamp_dataset)

## Count Number of Patients in the ward at each time stamp

In [58]:
mt_beds = 13
ds_beds = 5

In [ ]:
######### OLD SCRIPT ########
## Method took 448 seconds to run
#pcount_df['MT Patients'] = pcount_df.apply(lambda x: len(df[(df['Ward_Code'] == 'MT') & (df['Start'] <= x['Time']) & (df['End'] > x['Time'])]), axis=1)
#pcount_df['DS Patients'] = pcount_df.apply(lambda x: len(df[(df['Ward_Code'] == 'DS') & (df['Start'] <= x['Time']) & (df['End'] > x['Time'])]), axis=1)


In [ ]:
## TIMER FUNCTION
#start = time.time()
#end = time.time()
#print(end - start)

## Review Maternity Occupancy with Original Data
Split the data in two new frames for patients entering room & leaving room

In [95]:

## Make new dataframe patients entering room
mt_in_df= pd.DataFrame()
mt_in_df['Event'] = df.loc[df['Ward_Code'] == 'MT', 'Start']
mt_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
mt_out_df= pd.DataFrame()
mt_out_df['Event'] = df.loc[df['Ward_Code'] == 'MT', 'End']
mt_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
mt_count_df = pd.concat([mt_in_df, mt_out_df])
mt_count_df.sort_values('Event', ascending = True, inplace=True)
mt_count_df['MT_Patients'] = mt_count_df['Patient_Change'].cumsum()
mt_count_df['Bed_Available'] = mt_count_df['MT_Patients'] <= mt_beds
mt_count_df.dropna(inplace = True)
mt_count_df['MT_Patients'].describe()

count    11957.000000
mean         8.669482
std          2.515389
min          0.000000
25%          7.000000
50%          9.000000
75%         11.000000
max         15.000000
Name: MT_Patients, dtype: float64

In [ ]:
## Review Delivery Suite Occupance with Orginal Data

In [ ]:
# DELIVERY SUITE(DS)
## Make new dataframe patients entering room
ds_in_df = pd.DataFrame()
ds_in_df['Event'] = df.loc[df['Ward_Code'] == 'DS', 'Start']
ds_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
ds_out_df= pd.DataFrame()
ds_out_df['Event'] = df.loc[df['Ward_Code'] == 'DS', 'End']
ds_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
ds_count_df = pd.concat([ds_in_df, ds_out_df])
ds_count_df.sort_values('Event', ascending = True, inplace=True)
ds_count_df['DS_Patients'] = ds_count_df['Patient_Change'].cumsum()
ds_count_df['Bed_Available'] = ds_count_df['DS_Patients'] <= mt_beds
ds_count_df.dropna(inplace = True)
ds_count_df['DS_Patients'].describe()


## Describe the Results of patients occupying each Ward

In [10]:
#pcount_df.sort_values('DS Patients', ascending=False).head(5)
#pcount_df.set_index('Time', inplace = True)
pcount_df.describe()

## check DS for period of time during the year
#pcount_df['DS Patients'].between_time('9:00', '17:00').value_counts(normalize=True)*100


#print(41911 / (144704 + 41911))

,MT Patients,DS Patients
count,186615.000000,186615.000000
mean,8.687003,0.835469
std,2.804089,0.943369
min,0.000000,0.000000
25%,7.000000,0.000000
50%,9.000000,1.000000
75%,11.000000,1.000000
max,15.000000,5.000000


## Cross Check Check Results are correct

In [11]:
new_df = df.loc[(df['Ward_Code'] == 'MT') & (df['Start'] <= '2020-01-09 20:40:00') & (df['End'] > '2020-01-09 20:40:00')]
new_df

,Link,Ward_Code,Start,End
474,ADE-493053,MT,2020-01-09 14:00:00,2020-01-14 11:00:00
1529,ADE-492257,MT,2020-01-07 14:07:00,2020-01-12 10:06:00
2382,ADE-488422,MT,2020-01-07 15:40:00,2020-01-12 09:58:00
2737,ADE-485003,MT,2020-01-07 04:00:00,2020-01-11 10:15:00
3549,ADE-493109,MT,2020-01-09 11:30:00,2020-01-15 11:00:00
3758,ADE-492002,MT,2020-01-08 15:30:00,2020-01-11 12:52:00
4291,ADE-486372,MT,2020-01-09 19:06:00,2020-01-10 01:58:00
4355,ADE-485008,MT,2020-01-07 22:45:00,2020-01-11 10:56:00
4366,ADE-492774,MT,2020-01-07 14:06:00,2020-01-11 11:59:00
4392,ADE-484892,MT,2020-01-09 00:49:00,2020-01-14 10:00:00


## Create new ideal patient flow records with existing data and clean the dataset
In an ideal scenario, rather than sending patients to St Clares, St Helens and St Catherines they are sent to Maternity.  This process adjusts the actual data to what the ideal data would be.

As DPS is not relevant in this study we would also like to remove these entries from the dataset.

Once the data has been converted to ideal there are entries where the patient is transferred to the same room they left.  To clean the data we need to find the start date of when they first arrvied in the room and the end date when they left.

Once found we need to remove the duplicate entries and only show the first start date and last end date.

In [12]:
ideal_df = pd.DataFrame(df)

#CA, CL & HE Codes are converted to MT
ideal_df.loc[(ideal_df['Ward_Code'] == 'CA') | (ideal_df['Ward_Code'] == 'CL') | (ideal_df['Ward_Code'] == 'HE') , 'Ward_Code'] = 'MT' #Ward Code CA = MT

## Drop DPS from the new dataset
index_names = ideal_df.loc[(ideal_df['Ward_Code'] == 'DPS') | ((ideal_df['Ward_Code'] == 'ON'))].index
ideal_df.drop(index_names, inplace = True)

## Groupby the event code and with ward to summarise and fins any duplicates
events = ideal_df.groupby(['Link', 'Ward_Code'])
#events.get_group(('ADE-272865','MT'))

#get the min start date and max end date from the grouped results
ideal_df['min_start'] = events['Start'].transform('min')
ideal_df['max_end'] = events['End'].transform('max')

#Drop the Original start & end dates & rename the new calculated start & end
ideal_df.drop(columns=['Start', 'End'], inplace = True)
ideal_df.rename(columns= {'min_start':'Start', 'max_end': 'End'}, inplace = True)
ideal_df.sort_values(['Link', 'Ward_Code'], ascending = True, inplace = True)
ideal_df.drop_duplicates(keep='first', inplace = True)

#Calculate Length of Stay (LOS)
ideal_df['LOS'] = ideal_df['End'] - ideal_df['Start']

#checks that there are no longer any duplicates
#ideal_df.loc[ideal_df.duplicated(keep=False), :]
ideal_df

,Link,Ward_Code,Start,End,LOS
1,ADE-271165,MT,2013-06-30 22:21:00,2013-07-04 10:40:00,3 days 12:19:00
4,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00,0 days 21:40:00
5,ADE-271206,MT,2013-06-28 10:20:00,2013-07-03 09:59:00,4 days 23:39:00
12,ADE-271551,MT,2013-06-27 23:54:00,2013-07-01 09:25:00,3 days 09:31:00
14,ADE-271568,DS,2013-07-11 07:16:00,2013-07-12 13:59:00,1 days 06:43:00
...,...,...,...,...,...
2027,ADE-510071,DS,2020-07-18 09:51:00,2020-07-19 11:45:00,1 days 01:54:00
2029,ADE-510071,MT,2020-07-19 11:45:00,2020-07-23 11:50:00,4 days 00:05:00
1522,ADE-510160,DS,2020-07-20 22:00:00,2020-07-21 13:15:00,0 days 15:15:00
3060,ADE-510160,MT,2020-07-21 15:20:00,2020-07-25 10:45:00,3 days 19:25:00


## Show whether with the current bed amounts whether the patient would be accepted

In [19]:
beds_mt = 13
beds_ds = 5

def round_time(dt, i_min):
    a, b = divmod(dt.minute, i_min)
    c = i_min * (a + round(b / i_min, 0))
    hours, minutes= divmod(c, 60)
    hours = datetime.timedelta(hours = hours)
    minutes = datetime.timedelta(minutes = minutes)
    return datetime.datetime(dt.year, dt.month, dt.day, dt.hour, 0, 0) + hours + minutes

#ideal_df = ideal_df.apply(lambda x: )

pdate = '2019-07-18 09:51:00'
pdate = datetime.datetime.strptime(pdate, '%Y-%m-%d %H:%M:%S')
pdate = round_time(pdate, interval_period)

#ideal_df['Start_round'] = ideal_df.apply(lambda x: round_time(x['Start'], interval_period), axis=1)
ideal_df['MT_Beds'] = ideal_df.apply(lambda x: pcount_ideal_df[pcount_ideal_df['Time'] == round_time(x['Start'], interval_period)]['MT Patients'].squeeze() , axis=1)
ideal_df['DS_Beds'] = ideal_df.apply(lambda x: pcount_ideal_df[pcount_ideal_df['Time'] == round_time(x['Start'], interval_period)]['DS Patients'].squeeze() , axis=1)
ideal_df['MT_Beds_Available'] = ideal_df['MT_Beds'] <= beds_mt
ideal_df['DS_Beds_Available'] = ideal_df['DS_Beds'] <= beds_ds



#pcount_ideal_df[pcount_ideal_df['Time'] == ideal_df['Start']]['MT Patients']
#ideal_df['MT_Patients'] = ideal_df.apply(lambda x: pcount_ideal_df[pcount_ideal_df['Time'] == x['Start']]['MT Patients'], axis=1) #ideal_df.apply(lambda x: pcount_ideal_df.loc[pcount_ideal_df['Time'] == pdate]['MT Patients']) 
#ideal_df
#pcount_ideal_df.loc[pcount_ideal_df['Time']]

#ideal_df['No_of_Beds_Taken'] = pcount_ideal_df.apply(lambda x: )

In [20]:
ideal_df.loc[(ideal_df['Ward_Code'] != 'MT') | (ideal_df['Ward_Code'] != 'DS'), 'Beds_Available'] = True
ideal_df.loc[((ideal_df['Ward_Code'] == 'MT') & (ideal_df['MT_Beds_Available'] == True)) | ((ideal_df['Ward_Code'] == 'DS') & (ideal_df['DS_Beds_Available'] == True)), 'Beds_Available'] = True
ideal_df.loc[((ideal_df['Ward_Code'] == 'MT') & (ideal_df['MT_Beds_Available'] == False)) | ((ideal_df['Ward_Code'] == 'DS') & (ideal_df['DS_Beds_Available'] == False)), 'Beds_Available'] = False
ideal_df.loc[ideal_df['Ward_Code'] == 'DS', 'Beds_Available'].value_counts(normalize=True)*100
#ideal_df.drop(columns=['MT_Beds_Available','DS_Beds_Available','MT_Beds','DS_Beds'], inplace = True)

True    100.0
Name: Beds_Available, dtype: float64

In [21]:
ideal_df.sample(frac=0.1)

,Link,Ward_Code,Start,End,LOS,MT_Beds,DS_Beds,MT_Beds_Available,DS_Beds_Available,Beds_Available
5377,ADE-320106,MT,2015-02-12 23:30:00,2015-02-13 17:38:00,0 days 18:08:00,9,2,True,True,True
4446,ADE-313402,MT,2014-11-13 00:49:00,2014-11-18 10:20:00,5 days 09:31:00,10,1,True,True,True
7749,ADE-392888,MT,2017-05-03 13:58:00,2017-05-05 10:00:00,1 days 20:02:00,9,0,True,True,True
8038,ADE-340828,DS,2015-11-23 21:50:00,2015-11-24 04:30:00,0 days 06:40:00,7,1,True,True,True
5263,ADE-319095,MT,2014-09-04 12:30:00,2014-09-08 11:25:00,3 days 22:55:00,10,1,True,True,True
...,...,...,...,...,...,...,...,...,...,...
6321,ADE-327143,DS,2015-04-14 06:00:00,2015-04-15 00:40:00,0 days 18:40:00,9,1,True,True,True
6807,ADE-394394,MT,2017-08-03 11:13:00,2017-08-05 10:27:00,1 days 23:14:00,10,2,True,True,True
1065,ADE-382680,MT,2016-09-26 18:38:00,2016-09-27 11:23:00,0 days 16:45:00,8,1,True,True,True
6511,ADE-427372,DS,2018-06-24 20:45:00,2018-06-25 17:11:00,0 days 20:26:00,11,1,True,True,True


In [22]:
ideal_df.groupby('Ward_Code')['MT_Beds_Available'].value_counts()

Ward_Code  MT_Beds_Available
DS         True                 2158
           False                 181
ICU        True                   76
           False                   5
MT         True                 5449
           False                 335
Name: MT_Beds_Available, dtype: int64

## Review the Length of Stay (LOS) Results

In [23]:
ideal_df.groupby('Ward_Code').describe()

LOS                                                        \
          count                       mean                        std   
Ward_Code                                                               
DS         2339  0 days 22:19:06.618212911  0 days 12:09:48.296435046   
ICU          81  1 days 06:26:45.185185185  0 days 20:24:22.634704846   
MT         5783  3 days 23:55:35.929448383  1 days 15:17:05.855380638   

                                                                               \
                       min              25%              50%              75%   
Ward_Code                                                                       
DS         0 days 00:01:00  0 days 14:33:00  0 days 22:15:00  1 days 04:30:00   
ICU        0 days 02:34:00  0 days 17:53:00  0 days 23:59:00  1 days 15:17:00   
MT         0 days 01:30:00  3 days 04:25:00  3 days 20:38:00  4 days 18:36:00   

                            MT_Beds            ...             DS_Beds  \
                        max   count      mean  ...   75%   max   count   
Ward_Code                                      ...                       
DS          5 days 22:19:00  2339.0  9.262505  ...  12.0  18.0  2339.0   
ICU         6 days 15:52:00    81.0  9.123457  ...  12.0  18.0    81.0   
MT         34 days 19:50:00  5784.0  9.044606  ...  11.0  19.0  5784.0   

                                                        
               mean       std  min  25%  50%  75%  max  
Ward_Code                                               
DS         1.477127  1.030237  0.0  1.0  1.0  2.0  5.0  
ICU        0.716049  0.794270  0.0  0.0  1.0  1.0  3.0  
MT         0.992393  0.981558  0.0  0.0  1.0  2.0  5.0  

[3 rows x 24 columns]

## Create new dataframe and count the number of people in each Ward in an ideal Patient flow

In [15]:
pcount_ideal_df = pd.DataFrame(blank_timestamp_dataset)

In [16]:
pcount_ideal_df['MT Patients'] = pcount_ideal_df.apply(lambda x: len(ideal_df[(ideal_df['Ward_Code'] == 'MT') & (ideal_df['Start'] <= x['Time']) & (ideal_df['End'] > x['Time'])]), axis=1)
pcount_ideal_df['DS Patients'] = pcount_ideal_df.apply(lambda x: len(ideal_df[(ideal_df['Ward_Code'] == 'DS') & (ideal_df['Start'] <= x['Time']) & (ideal_df['End'] > x['Time'])]), axis=1)

## Summarise patient numbers in wards if they were not sent away

In [17]:
#pcount_ideal_df.sort_values('MT Patients', ascending=False).head(10)
pcount_ideal_df.sample(frac=0.1)

,Time,MT Patients,DS Patients
80440,2016-07-18 18:00:00,8,1
102319,2017-05-18 15:00:00,14,2
67897,2016-01-26 13:00:00,6,0
103285,2017-06-01 01:00:00,4,1
74583,2016-04-28 09:40:00,5,0
...,...,...,...
134796,2018-08-12 16:40:00,7,0
155070,2019-05-21 06:40:00,10,3
104653,2017-06-20 01:00:00,8,0
124678,2018-03-25 04:00:00,5,0


In [18]:
pcount_ideal_df.describe()

,MT Patients,DS Patients
count,186615.000000,186615.000000
mean,8.917456,0.840238
std,3.089753,0.946093
min,0.000000,0.000000
25%,7.000000,0.000000
50%,9.000000,1.000000
75%,11.000000,1.000000
max,20.000000,5.000000


## Work Hours at patient levels

In [24]:
pcount_ideal_df['MT Patients'].value_counts()

9     22583
8     22103
7     20628
10    20272
11    17717
12    16265
6     15716
13    15013
5     12368
4      8177
14     4030
3      3493
15     2227
16     1883
2      1624
17      905
1       825
18      569
19       88
0        76
20       53
Name: MT Patients, dtype: int64

In [25]:
pcount_ideal_df['DS Patients'].value_counts()

0    85610
1    58674
2    30593
3    10016
4     1717
5        5
Name: DS Patients, dtype: int64

# Adding Variability Information to the Transfers Data

In [63]:
## Extract Data fix columns
events_df = pd.read_csv('exception_dates.csv')
events_df.drop(columns=['Unnamed: 2'], inplace = True)
events_df.rename(columns = {'DATE':'Date', 'EVENT':'Event'}, inplace=True)
events_df['Date'] = events_df['Date'].apply(pd.to_datetime, format='%d/%m/%Y', errors='raise')
events_df['Date'] = events_df['Date'].dt.date
events_df.sort_values(['Event', 'Date'])

## Combine all labels with the same date into the same field

grouped_events_df = events_df.groupby(['Date'])['Event'].apply(','.join).reset_index()
grouped_events_df.sort_values('Event', ascending=True)

grouped_events_df.set_index('Date', inplace = True)
grouped_events_df
#grouped_events_df['Event'].value_counts()
#grouped_events_df['Count'] = 1
#grouped_events_df.groupby(['Event'], as_index=False)['Count'].count()



,Event
Date,
2012-01-02,PUBLIC
2012-01-26,PUBLIC
2012-03-12,PUBLIC
2012-04-06,"PUBLIC,SCHOOL"
2012-04-07,"PUBLIC,SCHOOL"
...,...
2021-01-22,SCHOOL
2021-01-23,SCHOOL
2021-01-24,SCHOOL


In [69]:
ideal_df['Date'] = ideal_df['Start'].dt.date
ideal_merged_df = pd.merge(grouped_events_df, ideal_df, how='right', on='Date')
ideal_merged_df['Event'].fillna('NORMAL', inplace=True)

ideal_merged_df['Event'].notnull().sum()

8204

In [72]:
ideal_merged_df.sample(frac=0.1).head(20)

,Date,Event,Link,Ward_Code,Start,End,LOS,MT_Beds,DS_Beds,MT_Beds_Available,DS_Beds_Available,Beds_Available
2745,2015-12-17,"SCHOOL,BEFOREXMAS",ADE-358261,MT,2015-12-17 10:00:00,2015-12-23 15:20:00,6 days 05:20:00,10,3,True,True,True
4572,2017-08-07,NORMAL,ADE-399761,MT,2017-08-07 21:01:00,2017-08-11 11:32:00,3 days 14:31:00,3,1,True,True,True
5701,2018-07-30,NORMAL,ADE-438702,DS,2018-07-30 00:01:00,2018-07-31 11:22:00,1 days 11:21:00,4,0,True,True,True
439,2013-12-21,"SCHOOL,BEFOREXMAS",ADE-284618,DS,2013-12-21 11:47:00,2013-12-22 13:30:00,1 days 01:43:00,11,2,True,True,True
6467,2019-03-18,NORMAL,ADE-463819,MT,2019-03-18 06:30:00,2019-03-23 11:00:00,5 days 04:30:00,4,0,True,True,True
4530,2017-07-30,NORMAL,ADE-398445,DS,2017-07-30 22:00:00,2017-08-01 06:49:00,1 days 08:49:00,9,1,True,True,True
7793,2020-03-31,NORMAL,ADE-498820,DS,2020-03-31 18:00:00,2020-04-01 00:54:00,0 days 06:54:00,15,2,False,True,True
4018,2017-03-08,NORMAL,ADE-382984,DS,2017-03-08 18:15:00,2017-03-09 11:18:00,0 days 17:03:00,9,3,True,True,True
2759,2016-01-16,SCHOOL,ADE-360270,MT,2016-01-16 12:30:00,2016-01-21 10:25:00,4 days 21:55:00,11,1,True,True,True
5138,2018-02-22,NORMAL,ADE-419309,MT,2018-02-22 18:09:00,2018-02-24 12:14:00,1 days 18:05:00,9,1,True,True,True


NameError: name 'ideal_merged_df' is not defined